In [ ]:
from source_files import *
from plot_helpers import *
from raster_compare.plots import PlotBase

import query_helpers

import math

# Analysis

In [ ]:
df = db_query_to_df()

In [ ]:
casi_colors = [CASI_COLORS[0], CASI_COLORS[2], CASI_COLORS[1]]
casi_classes = ['Snow', 'Rock', 'Vegetation']

# SfM measurement errors

In [ ]:
df.loc[df['casi_class'] == 'Water', 'casi_class'] = 'Vegetation'
negative_sfm = query_helpers.get_negative(df, 'sfm_snow_depth')

vegetation = (negative_sfm.casi_class == 'Vegetation')
open_areas = ~vegetation

open_areas = negative_sfm[open_areas]
vegetation = negative_sfm[vegetation]

In [ ]:
max_bin = math.fabs(math.floor(negative_sfm.sfm_snow_depth.min()))

sfm_bins = np.arange(-max_bin, 0, 0.1)
aso_bins = np.concatenate((
    np.arange(0, 2.0, 0.05),
    np.arange(2, 10.1, 0.10)
))
xticks = np.concatenate((
    np.arange(-40, 1, 2),
))

## ASO vs. SfM 

In [ ]:
matplotlib.rcParams['xtick.labelsize'] = LABEL_SIZE - 2
matplotlib.rcParams['ytick.labelsize'] = LABEL_SIZE - 2
annotate_args = dict(fontsize=14)

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(8, 4))

h1 = ax1.hist2d(
    negative_sfm['sfm_snow_depth'],
    negative_sfm['aso_snow_depth'],
    bins=[sfm_bins, aso_bins],
    cmap=HIST_2D_CMAP,
    vmin=0,
    vmax=1000,
    cmin=1,
)
ax1.set_facecolor('lightgrey')
ax1.tick_params(axis='both', length=6)
ax1.xaxis.set_ticks_position('bottom')

ax1.set_xlim(-40, 0)
ax1.set_xticks(xticks)
ax1.xaxis.set_minor_locator(MultipleLocator(1))
for label in ax1.xaxis.get_ticklabels()[::2]:
    label.set_visible(False)

ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
ax1.set_ylim(0, 5)
ax1.set_ylabel('ASO snow depth (m)', rotation=270, labelpad=14)

ax1.annotate('a)', xy=(-39.8, 0.8), **annotate_args)

cax = ax1.inset_axes([0.01, 0.89, 0.3, .08])
cax.yaxis.set_label_position('right')
cax.yaxis.set_label_coords(1.18, 0.9)
colorbar = fig.colorbar(h1[3], cax=cax, orientation='horizontal')
colorbar.ax.set_ylabel('Count', rotation=0)

insert_start = -4

bins = np.concatenate((
    [math.floor(negative_sfm.sfm_snow_depth.min())],
    np.arange(-40, 0.01, 0.01),
))
stack = []

for casi_class in casi_classes:
    stack.append(
        negative_sfm[negative_sfm.casi_class == casi_class].sfm_snow_depth
    )

stack_opts = dict(
    bins=bins,
    label=casi_classes,
    stacked=True,
    color=casi_colors,
    histtype='stepfilled',
    ec='black',
)
    
ax2.hist(stack, **stack_opts, linewidth=.3)
ax2.set_xlabel('SfM ' + SNOW_DEPTH_LABEL)
ax2.set_xlim(insert_start, bins.max())
PlotBase.format_axes_scientific(ax2, 'y', (4, 4), fontsize=11, labelpad=10)
PlotBase.move_yaxis_label_right(ax2)
ax2.annotate('b)', xy=(-3.98, 10000), **annotate_args)

ax3 = ax2.inset_axes([0.02, 0.4, 0.6, .6])
ax3.hist(stack, **stack_opts, linewidth=.1)
ax3.set_xlim(-30, insert_start)
ax3.set_xticks(np.arange(-28, -3, 4))
ax3.xaxis.set_minor_locator(MultipleLocator(1))
ax3.set_ylim(top=3000)
ax3.set_yticks(np.arange(0, 3000, 1000))
PlotBase.format_axes_scientific(ax3, 'y', (3, 3), fontsize=10, labelpad=11)
PlotBase.move_yaxis_label_right(ax3)
ax3.legend(loc='upper left', fontsize=10, frameon=False, labelspacing=.25);

## By Elevation 

### Data setup 

In [ ]:
elevation_bands = 10
elevation_min = df[df['elevation'] > 0].elevation.min()
elevation_min = elevation_min - elevation_min % elevation_bands
elevation_max = df.elevation.max() + (elevation_bands - df.elevation.max() % elevation_bands)
elevation_range = np.arange(elevation_min, elevation_max + elevation_bands, elevation_bands)

hist_opts = dict(
    vmin=0,
    vmax=1000,
    cmin=1,
)
bins = np.concatenate((
    np.arange(-max_bin, 0, 0.1),
    np.arange(0, 15.10, 0.10),
    np.arange(15, max_bin, 0.25),
))

hist_opts['bins'] = [bins, elevation_range]

### Plot setup

In [ ]:
figure_size = dict(figsize=(8, 2.75))
COLOR_BAR_ATTR = dict(right=0.9, rect=[0.91, 0.125, 0.03, 0.795])
HIST2D_PLOT = dict(ncols=2, sharey=True, **figure_size)

def plot_hist2d(ax, x_data, y_data, **kwargs):
    if 'label' in kwargs:
        ax.set_xlabel(kwargs['label'] + ' ' + SNOW_DEPTH_LABEL)
    if 'title' in kwargs:
        ax.set_title(kwargs['title'])
    ax.tick_params(axis='x', direction='inout', length=10)
    ax.tick_params(axis='y', length=6)
    ax.set_facecolor('lightgrey')
    ax.xaxis.set_ticks_position('both')
    
    data = ax.hist2d(x_data, y_data, cmap=HIST_2D_CMAP, **kwargs)
    
    ax.set_xlim(0, 40)

    return data

### Basin wide 

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    negative_sfm['sfm_snow_depth'],
    negative_sfm['elevation'],
    label='SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.set_xlim(-35, 0.2)
ax1.yaxis.set_ticks_position('both')

h2 = plot_hist2d(
    ax2,
    negative_sfm['aso_snow_depth'],
    negative_sfm['elevation'],
    label='ASO',
    **hist_opts,
)
ax2.set_xlim(right=15)

fig.suptitle('Snow Depth measurement errors by Elevation')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

### Open Areas

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    open_areas['sfm_snow_depth'],
    open_areas['elevation'],
    label='SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.set_xlim(-35, 0)

h2 = plot_hist2d(
    ax2,
    open_areas['aso_snow_depth'],
    open_areas['elevation'],
    label='ASO',
    **hist_opts,
)
ax2.set_xlim(right=15)

fig.suptitle('Snow Depth measurement errors by Elevation in open areas')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

### Vegetated areas

In [ ]:
fig, (ax1, ax2) = plt.subplots(**HIST2D_PLOT)

h1 = plot_hist2d(
    ax1,
    vegetation['sfm_snow_depth'],
    vegetation['elevation'],
    label='SfM',
    **hist_opts,
)
ax1.set_ylabel(ELEVATION_LABEL)
ax1.set_xlim(-35, 0)

h2 = plot_hist2d(
    ax2,
    vegetation['aso_snow_depth'],
    vegetation['elevation'],
    label='ASO',
    **hist_opts,
)
ax2.set_xlim(right=15)

fig.suptitle('Snow Depth measurement errors by Elevation in vegetated areas')
PlotBase.insert_colorbar(ax2, h2[3], 'count', **COLOR_BAR_ATTR);

## SfM by Aspect

### Data setup 

In [ ]:
bin_min = math.floor(negative_sfm.sfm_snow_depth.min())
bins = np.arange(0, bin_min - 0.1, -0.2)
bins = np.flip(bins)

aspect_range = np.arange(0, 361, 1)

hist_opts = dict(
    bins=[aspect_range, bins],
    vmin=0,
    vmax=200,
    cmin=1,
)

### Basin wide 

In [ ]:
fig = plt.figure(figsize=(8,4))
ax = fig.gca()
ax.set_facecolor('whitesmoke')
ha = ax.hist2d(
    negative_sfm['aspect'],
    negative_sfm['sfm_snow_depth'],
    **hist_opts,
)
PlotBase.insert_colorbar(ax, ha[3], 'Count')
ax.set_ylabel(SNOW_DEPTH_LABEL)
ax.set_xlabel('Aspect');

In [ ]:
aspect_stats = negative_sfm[['sfm_snow_depth', 'aspect']].groupby('aspect')

aspect_count = aspect_stats.count()
aspect_medians = aspect_stats.median()
aspect_means = aspect_stats.mean()

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(8, 16))
ax1.scatter(aspect_count.index, aspect_count.sfm_snow_depth)
ax1.set_title('Aspect count')

ax2.scatter(aspect_medians.index, aspect_medians.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference by aspect');

ax3.scatter(aspect_means.index, aspect_means.sfm_snow_depth)
ax3.set_xlim(aspect_range.min(), aspect_range.max())
ax3.set_ylabel('Mean SD difference')
ax3.set_xlabel('Aspect in degree')
ax3.set_title('Mean difference by aspect');

In [ ]:
aspect_count.sort_values('sfm_snow_depth', ascending=False).head(10).sort_index()

### Open Areas 

In [ ]:
open_areas_aspects = open_areas.groupby('aspect')
open_areas_aspects_median = open_areas_aspects.median()
open_areas_aspects_count = open_areas_aspects.count()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(
    3, 1, sharex=True, 
    figsize=(8,16), gridspec_kw={'hspace': 0.1}
)

ax1.set_facecolor('whitesmoke')

ha = ax1.hist2d(
    open_areas['aspect'],
    open_areas['sfm_snow_depth'],
    **hist_opts,
)
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('Negative values with aspect - Open Areas')

ax2.scatter(
    open_areas_aspects_median.index, 
    open_areas_aspects_median.sfm_snow_depth,
    s=2
)
ax2.set_title('Median difference by aspect - Open Areas')
ax2.axvline(HILLSHADE_SNOW_ON['azimuth'], color='goldenrod', linestyle='--')
ax2.annotate(
    'Snow On\n Sun Angle', 
    xy=(HILLSHADE_SNOW_ON['azimuth'] + 1, -0.15)
)
ax2.axvline(HILLSHADE_SNOW_ON['azimuth'] + 180, color='dimgrey', linestyle='--')
ax2.annotate(
    'Snow On\n Shade', 
    xy=(HILLSHADE_SNOW_ON['azimuth'] + 181, -0.5)
)
ax2.axvline(HILLSHADE_SNOW_FREE['azimuth'], color='goldenrod', linestyle='--')
ax2.annotate(
    'Snow Free\n Sun Angle', 
    xy=(HILLSHADE_SNOW_FREE['azimuth'] + 1, -0.5)
)
ax2.axvline(HILLSHADE_SNOW_FREE['azimuth'] % 180, color='dimgrey', linestyle='--')
ax2.annotate(
    'Snow Free\n Shade', 
    xy=((HILLSHADE_SNOW_FREE['azimuth'] + 181) % 180, -0.15)
)
ax2.set_ylim(top=0)
ax2.set_ylabel('Median SD difference')

ax3.scatter(
    open_areas_aspects_count.index, 
    open_areas_aspects_count.sfm_snow_depth,
    s=2
)
ax3.set_title('Count of aspects - Open Areas');
ax3.set_ylabel('Count')

ax3.set_xlim(aspect_range.min(), aspect_range.max())
ax3.set_xlabel('Aspect in degree');

## SfM by Slope

### Data setup 

In [ ]:
slope_range = np.arange(0, 91, 1)

hist_opts['bins'] = [slope_range, bins]

### Basin wide

In [ ]:
fig = plt.figure(figsize=(8,4))
ax = fig.gca()
ax.set_facecolor('whitesmoke')
hs = ax.hist2d(
    negative_sfm['slope'],
    negative_sfm['sfm_snow_depth'],
    **hist_opts
)
PlotBase.insert_colorbar(ax, hs[3], 'count')
ax.set_ylabel(SNOW_DEPTH_LABEL)
ax.set_xlabel('Slope')
ax.set_title('Negative values with slope');

In [ ]:
slope_stats = negative_sfm.groupby('slope')
slope_count = slope_stats.count()
slope_median = slope_stats.median()
slope_means = slope_stats.mean()

elevation_change_with_angle = [math.tan(math.radians(slope)) for slope in slope_count.index]

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(8, 16))

ax1.scatter(slope_count.index, slope_count.sfm_snow_depth)
ax1.set_title('Slope Count')

ax2.scatter(slope_median.index, slope_median.sfm_snow_depth)
ax2.set_ylabel('Median SD difference')
ax2.set_title('Median difference by slope')

ax3.scatter(slope_means.index, slope_means.sfm_snow_depth)
ax3.set_xlim(slope_range.min(), slope_range.max())
ax3.set_xlabel('Slope in degree')
ax3.set_ylabel('Mean SD difference')
ax3.set_title('Mean difference by slope');

### Open Areas 

In [ ]:
open_areas_slopes = open_areas.groupby('slope')
open_areas_slopes_median = open_areas_slopes.median()
open_areas_slopes_mean = open_areas_slopes.mean()
open_areas_slopes_count = open_areas_slopes.count()

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 1.75))

ax1.add_line(mlines.Line2D([0, 55], [-1, -1], linewidth=1, linestyle='--', color='darkorange'))
ax1.add_line(mlines.Line2D([55, 55], [-1.25, -20], linewidth=1, linestyle='dashdot', color='grey'))
for hl in [-4, -8, -12, -16]:
    ax1.axhline(hl, linewidth=0.25, linestyle='dotted', color='darkgrey')

ax1.plot(
    open_areas_slopes_median.index, 
    open_areas_slopes_median.sfm_snow_depth
)

ax1.xaxis.set_minor_locator(MultipleLocator(1))
ax1.set_xlabel('Slope (degrees)')
ax1.set_xlim(left=0)
ax1.set_xticks(np.arange(0, 91, 10))
ax1.set_ylim(-18, 0.2)
ax1.set_yticks(np.arange(0, -20, -4))
ax1.set_ylabel('Median Snow Depth \n Difference (m)');

#### Point statistics 

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(
    nrows=3, sharex=True, 
    figsize=(8,16), gridspec_kw={ 'hspace': 0.05}
)

ax1_data = ax1.plot(
    open_areas_slopes_median.index, 
    open_areas_slopes_median.sfm_snow_depth,
    color='deepskyblue',
    label='Median',
)
ax1.add_line(mlines.Line2D([0, 55], [-1, -1], linestyle='--', color='orange'))
ax1.add_line(mlines.Line2D([55, 55], [-1, -20], linestyle='--', color='orange'))
ax1.axvline(84, linestyle='--', color='orange')
ax1.xaxis.set_minor_locator(MultipleLocator(1))
ax1.set_ylim(-18, 0.2)
ax1.set_yticks(np.arange(0, -20, -2))
ax1.set_ylabel('SD difference')

ax1_2 = ax1.twinx()

ax1_2_data = ax1_2.plot(
    open_areas_slopes_count.index, 
    open_areas_slopes_count.sfm_snow_depth,
    color='indigo',
    label='Count',
)
ax1_2.set_xlabel('Slope in degree')
ax1_2.set_xlim(slope_range.min(), slope_range.max())
ax1_2.set_ylim(bottom=0)
ax1_2.set_ylabel('Count')

data = ax1_data + ax1_2_data
labels = [l.get_label() for l in data]
ax1.legend(data, labels);

ax2.plot(
    open_areas_slopes_median.index, 
    open_areas_slopes_median.sfm_point_count_snow_free, 
    label='Median Point Count Snow Free'
)
ax2.plot(
    open_areas_slopes_median.index, 
    open_areas_slopes_median.sfm_point_count_snow_on, 
    label='Median Point Count Snow On'
)
ax2.axvline(55, linestyle='--', color='orange')
ax2.axvline(84, linestyle='--', color='orange')
ax2.set_ylim(bottom=-.2)
ax2.set_ylabel('Point Count')
ax2.legend();

ax3.plot(
    open_areas_slopes_median.index, 
    elevation_change_with_angle,
    label=r'Max $\Delta$m in pixel (theoretical)'
)
ax3.plot(
    open_areas_slopes_median.index, 
    open_areas_slopes_median.point_spread_snow_on,
    label='Median Point Spread Snow On'
)
ax3.plot(
    open_areas_slopes_median.index, 
    open_areas_slopes_median.point_spread_snow_free,
    label='Median Point Spread Snow Free'
)
ax3.axvline(55, linestyle='--', color='orange')
ax3.axvline(84, linestyle='--', color='orange')
ax3.set_ylabel('Meter')
ax3.set_xlabel('Slope Angle')
ax3.legend()

plt.show()

### ASO all values basin wide

In [ ]:
bin_max = math.ceil(df.aso_snow_depth.max())
bins = np.arange(0, bin_max + 0.1, 0.1)

hist_opts['bins'] = [slope_range, bins]

fig, ax1 = plt.subplots(
    figsize=(13, 6),
    dpi=150,
)
ax1.set_facecolor('whitesmoke')
ha = ax1.hist2d(
    df.slope,
    df.aso_snow_depth,
    **hist_opts,
)
ax1.set_ylabel(SNOW_DEPTH_LABEL)
ax1.set_title('ASO snow depth by slope');

# Point Density in Vegetation 

In [ ]:
negative_sfm[['sfm_point_count_snow_free', 'sfm_point_count_snow_on', 'point_spread_snow_free' , 'point_spread_snow_on']].agg([np.mean, np.median, np.std])